In [1]:
from pathlib import Path
import sys  
import os

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
from libs.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
import numpy as np
import pandas as pd

In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM
from libs.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [5]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [6]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 157 ( 0.4%) Users that have less than 1 test interactions


In [7]:
URM_all_coo = URM_all.tocoo()

slim_model = SLIMElasticNetRecommender(URM_all_coo, verbose=True)
slim_model.fit(l1_ratio=0.15873684210526315, alpha = 0.00034, topK = 20)


SLIMElasticNetRecommender: Processed 6650 (17.4%) in 5.00 min. Items per second: 22.16
SLIMElasticNetRecommender: Processed 13848 (36.3%) in 10.00 min. Items per second: 23.08
SLIMElasticNetRecommender: Processed 21158 (55.5%) in 15.00 min. Items per second: 23.50
SLIMElasticNetRecommender: Processed 28532 (74.8%) in 20.00 min. Items per second: 23.77
SLIMElasticNetRecommender: Processed 36029 (94.5%) in 25.00 min. Items per second: 24.02
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 26.51 min. Items per second: 23.97


In [8]:
result_dict, _ = evaluator_validation.evaluateRecommender(slim_model)

EvaluatorHoldout: Processed 35579 (100.0%) in 10.30 sec. Users per second: 3453


In [9]:
result_dict["MAP"][10]

0.0

In [10]:
def write_submission_lib_model(trained_model, filename: str = "submission.csv") -> None:
	"""Builds the submission file from a trained recommender model. The file is saved in a CSV format.

	:param trained_model: A fitted recommender model
	:type trained_model: RecommenderModel
	:param filename: The filename of the submission for this particular recommender model
	:type filename: str
	"""
	target_users_test = pd.read_csv("../data/data_target_users_test.csv",).to_numpy().ravel()

	recommendations = np.array([
		trained_model.recommend(user_id, cutoff=10) for user_id in target_users_test
	])

	if not os.path.exists("../submissions"):
		os.makedirs("../submissions")
	with open(f"../submissions/{filename}", "w") as f:
		f.write("user_id,item_list\n")
		for user_id, recs in zip(target_users_test, recommendations):
			f.write(f"{user_id},{' '.join(map(str, recs))}\n")

In [11]:
write_submission_lib_model(slim_model, filename='slim_urm2.csv')